In [1]:
import pandas as pd

### Caricamento dataframe elaborati

In [2]:
filtered_european_yearly_energy = pd.read_csv("../data/processed/filtered_european_yearly_energy.csv")
european_wholesale_prices = pd.read_csv("../data/processed/european_wholesale_prices.csv")

In [3]:
filtered_european_yearly_energy.columns

Index(['Area', 'ISO 3 code', 'Year', 'Demand', 'Net Imports',
       'Total Generation', 'Clean', 'Nuclear', 'Wind and Solar',
       'Hydro, Bioenergy and Other Renewables', 'Fossil', 'Coal',
       'Gas and Other Fossil'],
      dtype='object')

In [4]:
european_wholesale_prices.columns

Index(['Country', 'ISO3 Code', 'Date', 'Price (EUR/MWhe)'], dtype='object')

Uniformazione colonna `IS03 Code`

In [5]:
filtered_european_yearly_energy = filtered_european_yearly_energy.rename(
    columns={"ISO 3 code": "ISO3 Code"}
)

In [6]:
european_wholesale_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3739 entries, 0 to 3738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           3739 non-null   object 
 1   ISO3 Code         3739 non-null   object 
 2   Date              3739 non-null   object 
 3   Price (EUR/MWhe)  3739 non-null   float64
dtypes: float64(1), object(3)
memory usage: 117.0+ KB


In [7]:
european_wholesale_prices["Date"] = pd.to_datetime(
    european_wholesale_prices["Date"],
    format="%Y-%m-%d"
)

Bisogna calcolare i prezzi annuali per poter aggregare i dati con il dataset della produzione energetica

Dei valori origianli estrapoliamo:
- la media -> indica l'andamento medio dei prezzi della corrente nella determintà nazione
- il valore massimo -> può indicare un momento di difficoltà del mercato elettrico (scaristà di offerta, crisi, ...)
- deviazione standard -> ci aiuta a capire se il sistema è stabile o meno

In [8]:
european_prices_yearly = (
    european_wholesale_prices.assign(Year= lambda df: df["Date"].dt.year)
    .groupby(["ISO3 Code", "Year"], as_index=False)
    .aggregate(
            AvgPrice_EUR_MWh=("Price (EUR/MWhe)", "mean"),
            MaxPrice_EUR_MWh=("Price (EUR/MWhe)", "max"),
            StdPrice_EUR_MWh=("Price (EUR/MWhe)", "std"),
        )
)

In [9]:
european_prices_yearly

,ISO3 Code,Year,AvgPrice_EUR_MWh,MaxPrice_EUR_MWh,StdPrice_EUR_MWh
0,AUT,2015,31.764167,39.37,3.817206
1,AUT,2016,28.956667,38.21,5.792114
2,AUT,2017,34.419167,52.41,6.841540
3,AUT,2018,40.891667,48.63,6.231735
4,AUT,2019,40.137500,56.68,6.267981
...,...,...,...,...,...
312,SWE,2021,64.228333,156.77,33.719934
313,SWE,2022,119.453333,235.20,55.289983
314,SWE,2023,51.834167,80.98,20.836311
315,SWE,2024,36.121667,68.66,18.064213


#### Unione dei due dataset
L'unione avverà tramite codice IS03 e l'anno dell'osservazione <br>
Il metodo di unione è inner su `filtered_european_yearly_energy`

In [10]:
european_energy = filtered_european_yearly_energy.merge(
    european_prices_yearly,
    on=["ISO3 Code", "Year"],
    how="inner"
)

In [11]:
european_energy.head()

,Area,ISO3 Code,Year,Demand,Net Imports,Total Generation,Clean,Nuclear,Wind and Solar,"Hydro, Bioenergy and Other Renewables",Fossil,Coal,Gas and Other Fossil,AvgPrice_EUR_MWh,MaxPrice_EUR_MWh,StdPrice_EUR_MWh
0,Austria,AUT,2015,71.84,10.06,61.78,47.59,0.0,5.78,41.81,14.19,2.95,11.24,31.764167,39.37,3.817206
1,Austria,AUT,2016,72.03,7.16,64.87,51.08,0.0,6.33,44.75,13.79,2.04,11.75,28.956667,38.21,5.792114
2,Austria,AUT,2017,73.64,6.55,67.09,51.05,0.0,7.84,43.21,16.04,1.76,14.28,34.419167,52.41,6.841540
3,Austria,AUT,2018,73.62,8.95,64.67,50.06,0.0,7.49,42.57,14.61,1.81,12.80,40.891667,48.63,6.231735
4,Austria,AUT,2019,73.82,3.13,70.69,54.64,0.0,9.15,45.49,16.05,1.50,14.55,40.137500,56.68,6.267981


#### Controllo duplicazione delle chiavi

In [12]:
(
    european_energy.duplicated(["ISO3 Code", "Year"]).sum(),
    filtered_european_yearly_energy.duplicated(["ISO3 Code", "Year"]).sum(),
    european_prices_yearly.duplicated(["ISO3 Code", "Year"]).sum()
)

(np.int64(0), np.int64(0), np.int64(0))

#### Controllo valori mancanti

In [13]:
european_energy.isna().sum().sort_values(ascending=False)

StdPrice_EUR_MWh                         1
Area                                     0
Year                                     0
ISO3 Code                                0
Net Imports                              0
Total Generation                         0
Clean                                    0
Demand                                   0
Nuclear                                  0
Wind and Solar                           0
Fossil                                   0
Hydro, Bioenergy and Other Renewables    0
Coal                                     0
Gas and Other Fossil                     0
AvgPrice_EUR_MWh                         0
MaxPrice_EUR_MWh                         0
dtype: int64

L'unico valore mancante è la deviaizone standard del prezzo per la Serbia nel 2016, questo la mancanza di dati (media e massimo coincidono)
Mantengo il valore a NaN per non alterare il dataset e dare un informazione fuorvinate, quando farò delle analisi sulle varie volatilità del prezzo escluderò questo valore.

In [14]:
european_energy[
    european_energy["StdPrice_EUR_MWh"].isna()
    ]

,Area,ISO3 Code,Year,Demand,Net Imports,Total Generation,Clean,Nuclear,Wind and Solar,"Hydro, Bioenergy and Other Renewables",Fossil,Coal,Gas and Other Fossil,AvgPrice_EUR_MWh,MaxPrice_EUR_MWh,StdPrice_EUR_MWh
218,Serbia,SRB,2016,36.69,-1.92,38.61,10.86,0.0,0.04,10.82,27.75,27.24,0.51,52.88,52.88,NaN


### Salvataggio in data/processed

In [15]:
european_energy.to_csv("../data/processed/european_yearly_energy_mix_and_prices.csv", index=False)